In [10]:
##  Import packages and set seed

import numpy as np
import pandas as pd
from numpy.random import seed
from numpy.random import randint
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix , classification_report
seed(1)

### Build our Dataset
For the dataset, we will generate it randomly. I include a simple code to make the dataset. Test dataset will be generated after our model is ready.

In [11]:
## Initiate data frame
d = {'x': [], 'x1': [],'x2': [], 'x3': [],'x4': [], 'x5': [],'geom':[],'aritmatik':[],'none':[]}
df = pd.DataFrame(data=d)
df

,x,x1,x2,x3,x4,x5,geom,aritmatik,none


In [12]:
## generate random data
for i in range(500):
    k = randint(3)
    if k == 0:
        #generate geom
        a = randint(low = -200, high= 200)
        r = randint(low= -10,high=10)
        data = {'x':a, 'x1': a*r,'x2': a*r**2, 'x3': a*r**3,'x4': a*r**4, 'x5': a*r**5,'geom':1,'aritmatik':0,'none':0}
        df = df.append(data,ignore_index=True)
        
    elif k == 1:
        #generate aritmatik
        a = randint(low = -200, high= 200)
        b = randint(low= -50,high=50)
        data = {'x':a, 'x1': a+b,'x2': a+b*2, 'x3': a+b*3,'x4': a+b*4, 'x5': a+b*5,'geom':0,'aritmatik':1,'none':0}
        df = df.append(data,ignore_index=True)
    
    else:
        #generate geom
        a = randint(low = -1000, high= 1000,size =6)
        data = {'x':a[0], 'x1': a[1],'x2': a[2], 'x3': a[3],'x4': a[4], 'x5': a[5],'geom':0,'aritmatik':0,'none':1}
        df = df.append(data,ignore_index=True)

In [13]:
## Our Dataset summary

print(df.info())
df[['aritmatik','geom','none']].sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   x          500 non-null    float64
 1   x1         500 non-null    float64
 2   x2         500 non-null    float64
 3   x3         500 non-null    float64
 4   x4         500 non-null    float64
 5   x5         500 non-null    float64
 6   geom       500 non-null    float64
 7   aritmatik  500 non-null    float64
 8   none       500 non-null    float64
dtypes: float64(9)
memory usage: 35.3 KB
None


aritmatik    158.0
geom         172.0
none         170.0
dtype: float64

In [14]:
## Make it to X_train and y_train

X_train = df[['x','x1','x2','x3','x4','x5']].to_numpy()
y_train = df[['geom','aritmatik','none']].to_numpy()

In [15]:
df

,x,x1,x2,x3,x4,x5,geom,aritmatik,none
0,35.0,-3.0,-41.0,-79.0,-117.0,-155.0,0.0,1.0,0.0
1,55.0,-55.0,55.0,-55.0,55.0,-55.0,1.0,0.0,0.0
2,135.0,149.0,163.0,177.0,191.0,205.0,0.0,1.0,0.0
3,-71.0,-142.0,-284.0,-568.0,-1136.0,-2272.0,1.0,0.0,0.0
4,190.0,165.0,140.0,115.0,90.0,65.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
495,-11.0,55.0,-275.0,1375.0,-6875.0,34375.0,1.0,0.0,0.0
496,-310.0,-430.0,651.0,487.0,934.0,896.0,0.0,0.0,1.0
497,196.0,157.0,118.0,79.0,40.0,1.0,0.0,1.0,0.0
498,521.0,442.0,798.0,-382.0,733.0,274.0,0.0,0.0,1.0


In [16]:
W1 = np.random.randn(10, X_train.shape[1]) * 0.01 ## (our first weight consist of (# nodes of target later,# nodes input))
b1 = np.zeros(shape =(10, 1)) 
  
W2 = np.random.randn(y_train.shape[1], 10) * 0.01 ## (our second weight(# target nodes, # previous layer nodes))
b2 = np.zeros(shape =(y_train.shape[1], 1)) 

In [17]:
print(W1.shape)
print(b1.shape)
print(W2.shape)
print(b2.shape)

(10, 6)
(10, 1)
(3, 10)
(3, 1)


### Softmax Activation
I use Softmax activation because our model has 3 targets classification task.
To keep it simple the sigmoid function is used for the two-class logistic regression, whereas the softmax function is used for the multiclass logistic regression.

[Quick look to a math behind softmax](https://www.youtube.com/watch?v=o6HrH2EMD-w) 

In [18]:
def softmax(x):
    z = np.exp(x)
    zsum = z.sum(axis=0)
    k = np.zeros(shape = z.shape)
    for i in range(len(zsum)):
        k[:,i] = z[:,i]/zsum[i]
    return k

#### Forward propagation
I use matrix operator to perform our forward and later backward propagation.

[Based from this](https://www.geeksforgeeks.org/deep-neural-net-with-forward-and-back-propagation-from-scratch-python/)

In [19]:
def forward_prop(X, W1, W2, b1, b2): 
  
    Z1 = np.dot(W1, X.T) + b1 
    A1 = np.tanh(Z1) 
    Z2 = np.dot(W2, A1) + b2 
    A2 = softmax(Z2) 
      
    # cache that will be used for back propagation
    cache = {"Z1": Z1, 
             "A1": A1, 
             "Z2": Z2, 
             "A2": A2} 
      
    return A2, cache 

#### Loss function
For our loss function we use cross entrophy function

[for deeper loss function understanding](https://gombru.github.io/2018/05/23/cross_entropy_loss/)

In [20]:
# Crossentropy
# Here Y is actual output 
def cost_crossentropy(A2, Y): 
    ## Implementing cross entrophy function for multiclass problem
    cost = -np.mean(Y * np.log(A2.T + 1e-8))
      
    return cost

#### Back propagation
For back propagation, normal backpropagation with gradient descent wont work, our model parameter trapped at some local minima.
So I apply adam optimizer on backpropagation process

[Math Behind Adam Optimizer](https://arxiv.org/abs/1412.6980) (a paper)

In [21]:
def back_propagate(W1, b1, W2, b2, cache,cache2,Y,t,learning_rate = 0.01): 
    beta1 = 0.9
    beta2 = 0.999
    # Retrieve also A1 and A2 from dictionary "cache" 
    A1 = cache['A1'] 
    A2 = cache['A2']
    # Retrieve m_dw, v_dw, m_db, v_db from dictionary "cache2"
    ## Cache2 is needed to backpropagate with adam optimizer
    m_dw2 = cache2['m_dw2']
    m_db2 = cache2['m_db2']
    v_dw2 = cache2['v_dw2']
    v_db2 = cache2['v_db2']
    m_dw1 = cache2['m_dw1']
    m_db1 = cache2['m_db1']
    v_dw1 = cache2['v_dw1']
    v_db1 = cache2['v_db1']
    
    #Layer 2
  
    # Backward propagation: calculate dW1, db1, dW2, db2
    m = Y.shape[0]
    dZ2 = A2 - Y.T
    dW2 = (1 / m) * np.dot(dZ2, A1.T) 
    db2 = (1 / m) * np.sum(dZ2, axis = 1, keepdims = True)
    
    cache2['m_dw2'] = beta1*m_dw2 + (1-beta1)*dW2
    cache2['m_db2'] = beta1*m_db2 + (1-beta1)*db2
    cache2['v_dw2'] = beta2*v_dw2 + (1-beta2)*(np.multiply(dW2,dW2))
    cache2['v_db2'] = beta2*v_db2 + (1-beta2)*(np.multiply(db2,db2))
    
    m_dw_corr2 = cache2['m_dw2']/(1-beta1**t)
    m_db_corr2 = cache2['m_db2']/(1-beta1**t)
    v_dw_corr2 = cache2['v_dw2']/(1-beta2**t)
    v_db_corr2 = cache2['v_db2']/(1-beta2**t)
    
    ## update weights and biases
    W2 = W2 - learning_rate *(m_dw_corr2/(np.sqrt(v_dw_corr2)+1e-8))
    b2 = b2 - learning_rate *(m_db_corr2/(np.sqrt(v_db_corr2)+1e-8))  ### line 37
    
    
    ## Layer 1
  
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2)) 
    dW1 = (1 / m) * np.dot(dZ1, X_train) 
    db1 = (1 / m) * np.sum(dZ1, axis = 1, keepdims = True)
    
    cache2['m_dw1'] = beta1*m_dw1 + (1-beta1)*dW1
    cache2['m_db1'] = beta1*m_db1 + (1-beta1)*db1
    cache2['v_dw1'] = beta2*v_dw1 + (1-beta2)*(np.multiply(dW1,dW1))
    cache2['v_db1'] = beta2*v_db1 + (1-beta2)*(np.multiply(db1,db1))
    
    m_dw_corr1 = cache2['m_dw1']/(1-beta1**t)
    m_db_corr1 = cache2['m_db1']/(1-beta1**t)
    v_dw_corr1 = cache2['v_dw1']/(1-beta2**t)
    v_db_corr1 = cache2['v_db1']/(1-beta2**t)
    
    W1 = W1 - learning_rate *(m_dw_corr1/(np.sqrt(v_dw_corr1)+1e-8))
    b1 = b1 - learning_rate *(m_db_corr1/(np.sqrt(v_db_corr1)+1e-8))   ### line 57
  
    return W1, W2, b1, b2 

#### Fit our data

Our model architecture  is ready, we need to fit our data to train our model.
Apparently, ANN model with only one hidden layer still can't catch complexity for data with extreme outliers. If we take a quick look to our dataset. We can found some data that multiply greatly or randomly high and low. That make the input dynamics pretty complex. So I need to put some scaler. Because of extreme outliers, I use Robust scaler that take 25th, 50th and 75th percentile of our data to rescaling our data instead of standard deviation.

[Robust Scaling Formula](https://machinelearningmastery.com/robust-scaler-transforms-for-machine-learning/#:~:text=Standardization%20is%20calculated%20by%20subtracting,dividing%20by%20the%20standard%20deviation.&text=Sometimes%20an%20input%20variable%20may%20have%20outlier%20values.&text=This%20is%20called%20robust%20standardization,the%2025th%20and%2075th%20percentiles)

In [22]:
scalar = RobustScaler()
scalar.fit(X_train)
X_train = scalar.transform(X_train)

In [23]:
# Please note that the weights and bias are global  
# Here num_iteration is epochs
# Cache 2 are essensial to train our data with Adam Optimizer
array_cost = np.array([])
cache2 = {'m_dw2':0,
          'm_db2':0,
          'v_dw2':0,
          'v_db2':0,
          'm_dw1':0,
          'm_db1':0,
          'v_dw1':0,
          'v_db1':0}

for i in range(0,501):
    t = i +1
        
    # Forward propagation. Inputs: "X, parameters". return: "A2, cache". 
    A2, cache = forward_prop(X_train, W1, W2, b1, b2) 
          
    # Cost function. Inputs: "A2, Y". Outputs: "cost". 
    cost = cost_crossentropy(A2, y_train)
   
    # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads". 
    W1, W2, b1, b2 = back_propagate(W1, b1, W2, b2, cache,cache2,y_train,t,0.01) 
          
    # Print the cost every 1000 iterations 
    if i % 50 == 0: 
        print ("Cost after iteration % i: % f" % (i, cost))
            
    array_cost = np.append(array_cost,cost)

Cost after iteration  0:  0.366575
Cost after iteration  50:  0.281343
Cost after iteration  100:  0.195156
Cost after iteration  150:  0.116151
Cost after iteration  200:  0.086671
Cost after iteration  250:  0.075801
Cost after iteration  300:  0.069707
Cost after iteration  350:  0.064787
Cost after iteration  400:  0.060253
Cost after iteration  450:  0.055362
Cost after iteration  500:  0.049654


# Model with test dataset

In [24]:
## Initiate data frame
p = {'x': [], 'x1': [],'x2': [], 'x3': [],'x4': [], 'x5': [],'geom':[],'aritmatik':[],'none':[]}
df_try = pd.DataFrame(data=p)

for i in range(100):
    k = randint(3)
    if k == 0:
        #generate geom
        a = randint(low = -200, high= 200)
        r = randint(low= -10,high=10)
        data = {'x':a, 'x1': a*r,'x2': a*r**2, 'x3': a*r**3,'x4': a*r**4, 'x5': a*r**5,'geom':1,'aritmatik':0,'none':0}
        df_try = df_try.append(data,ignore_index=True)
        
    elif k == 1:
        #generate aritmatik
        a = randint(low = -200, high= 200)
        b = randint(low= -50,high=50)
        data = {'x':a, 'x1': a+b,'x2': a+b*2, 'x3': a+b*3,'x4': a+b*4, 'x5': a+b*5,'geom':0,'aritmatik':1,'none':0}
        df_try = df_try.append(data,ignore_index=True)
    
    else:
        #generate geom
        a = randint(low = -1000, high= 1000,size =6)
        data = {'x':a[0], 'x1': a[1],'x2': a[2], 'x3': a[3],'x4': a[4], 'x5': a[5],'geom':0,'aritmatik':0,'none':1}
        df_try = df_try.append(data,ignore_index=True)

In [25]:
X_try = df_try[['x','x1','x2','x3','x4','x5']].to_numpy()
y_test = df_try[['geom','aritmatik','none']].to_numpy()
X_test = scalar.transform(X_try)

In [26]:
## Predict
pred, cache = forward_prop(X_test, W1, W2, b1, b2)

In [33]:
print(df_try[['geom','aritmatik','none']].head(5))
pred_df = pd.DataFrame(data = pred.T,columns = ['Geom','Aritmatik','None'])
pred_df.head(5)

   geom  aritmatik  none
0   1.0        0.0   0.0
1   0.0        0.0   1.0
2   0.0        0.0   1.0
3   1.0        0.0   0.0
4   1.0        0.0   0.0


,Geom,Aritmatik,None
0,0.972910,0.008871,0.018219
1,0.002454,0.000027,0.997518
2,0.002512,0.015575,0.981913
3,0.982689,0.010167,0.007144
4,0.982689,0.010167,0.007144


#### Model Evaluation

I provide some confusion matrix and classification report to get a big picture of our model performance

In [34]:
## Confusion Matrix train

y_train_ind = np.argmax(y_train,axis =1)
y_predict_tr,cache_tr = forward_prop(X_train, W1, W2, b1, b2)
y_preds_train = np.argmax(y_predict_tr.T,axis =1)
cm_train = confusion_matrix(y_train_ind,y_preds_train)

In [35]:
cm_train

array([[152,  16,   4],
       [  0, 158,   0],
       [  0,   0, 170]], dtype=int64)

In [36]:
print('classification report train')
print(classification_report(y_train_ind, y_preds_train, target_names=['Geom','Aritmatik','none']))

classification report train
              precision    recall  f1-score   support

        Geom       1.00      0.88      0.94       172
   Aritmatik       0.91      1.00      0.95       158
        none       0.98      1.00      0.99       170

    accuracy                           0.96       500
   macro avg       0.96      0.96      0.96       500
weighted avg       0.96      0.96      0.96       500



In [37]:
## Confusion Matrix test

y_test_ind = np.argmax(y_test,axis =1)
y_predict_test,cache_test = forward_prop(X_test, W1, W2, b1, b2)
y_preds_test = np.argmax(y_predict_test.T,axis =1)
cm_test = confusion_matrix(y_test_ind,y_preds_test)

In [38]:
cm_test

array([[38,  1,  1],
       [ 0, 28,  0],
       [ 2,  1, 29]], dtype=int64)

In [39]:
print('classification report train')
print(classification_report(y_test_ind, y_preds_test, target_names=['Geom','Aritmatik','none']))

classification report train
              precision    recall  f1-score   support

        Geom       0.95      0.95      0.95        40
   Aritmatik       0.93      1.00      0.97        28
        none       0.97      0.91      0.94        32

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100



The report itself, based on f1-score both on train and test set. The model already get some satisfying results

### Read csv

Use our model to some a dataset that I prepare to compare our model with the same exact model but with Keras package(tf backend)

In [40]:
csv = pd.read_csv('compare.csv',header = 0)

In [41]:
csv

,x,x1,x2,x3,x4,x5,geom,aritmatik,none
0,16.0,-16.0,-48.0,-80.0,-112.0,-144.0,0.0,1.0,0.0
1,381.0,-19.0,234.0,815.0,-864.0,233.0,0.0,0.0,1.0
2,-27.0,54.0,-108.0,216.0,-432.0,864.0,1.0,0.0,0.0
3,98.0,882.0,7938.0,71442.0,642978.0,5786802.0,1.0,0.0,0.0
4,-58.0,116.0,-232.0,464.0,-928.0,1856.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
70,-150.0,300.0,-600.0,1200.0,-2400.0,4800.0,1.0,0.0,0.0
71,99.0,101.0,103.0,105.0,107.0,109.0,0.0,1.0,0.0
72,636.0,-695.0,-714.0,314.0,-287.0,134.0,0.0,0.0,1.0
73,129.0,-247.0,62.0,263.0,-533.0,954.0,0.0,0.0,1.0


In [42]:
X_csv = csv[['x','x1','x2','x3','x4','x5']].to_numpy()
y_csv = csv[['geom','aritmatik','none']].to_numpy()
X_csv = scalar.transform(X_csv)

In [43]:
## Predict
pred_csv, cache = forward_prop(X_test, W1, W2, b1, b2)

In [44]:
## Confusion Matrix test
y_csv_ind = np.argmax(y_csv,axis =1)
y_predict_csv,cache_test = forward_prop(X_csv, W1, W2, b1, b2)
y_preds_csv = np.argmax(y_predict_csv.T,axis =1)
cm_csv = confusion_matrix(y_csv_ind,y_preds_csv)

In [45]:
cm_csv

array([[25,  3,  0],
       [ 0, 27,  0],
       [ 3,  0, 17]], dtype=int64)

In [46]:
print('classification report train')
print(classification_report(y_csv_ind, y_preds_csv, target_names=['Geom','Aritmatik','none']))

classification report train
              precision    recall  f1-score   support

        Geom       0.89      0.89      0.89        28
   Aritmatik       0.90      1.00      0.95        27
        none       1.00      0.85      0.92        20

    accuracy                           0.92        75
   macro avg       0.93      0.91      0.92        75
weighted avg       0.92      0.92      0.92        75



In [47]:
csv[['geom','aritmatik','none']].sum()

geom         28.0
aritmatik    27.0
none         20.0
dtype: float64